#  D:\redpitaya\tmp\3_stopwatch_modif\readFIFO_ADC_lineabase_dinamic_32muestras\


#Le damos permisos de modificación al archivo readfifo.bit
#!chmod 755 readfifo_linebase_32.bit
#!ls
--                   B7%B6              B5        B4        B3      B2       B1          B0

                                 nrst      cntrl_clk              rd_en_out wr_en_out
                                       
leds <=  ADC_counter(24 downto 23) & gpioIN(3) & gpioIN(2) & empty & full & gpioIN(1) & gpioIN(0);


# Proyecto vivado localizado en D:\redpitaya\tmp\3_stopwatch_modif\readFIFO_ADC_lineabase_dinamic_32muestras\

scp .\system_wrapper.bit root@192.168.100.10:/root/RedPitaya/readfifo_linebaseDinamic.bit

# Para el copiado del archivo a rp es 
# scp readFIFO_64.bit root@rp-f04e41:~/RedPitaya

# Dar permisos de editor con chmod 777 para poder activar el archivo en 
# terminal

# archivo bitstream read_FIFO64muestras.bit, localizado en /root/RedPitaya
## ejecutar la operacion siguiente antes del inicio de ejecución del código python

## cat  readFIFO_ADC64muestras.bit > /dev/xdevcfg

## se ejecuta el programa a 64 muestras y con un buffer de 512 datos en vez de 1024

## wr_en_out <= gpioIN(0);
## rd_en_out <= gpioIN(1);
## cntrl_clk    <= gpioIN(2);
## nrst           <= gpioIN(3);
## --               <= gpioIN(4);   Control modo de adquisición
## --               <= gpioIN(5);   Control modo de adquisición
## --               <= gpioIN(6);
## --               <= gpioIN(7);

| gpioIN(5-4) | ADC_FIFO    || clock_FIFO    | Description   |
|-------------|-------------||---------------|---------------|
|   ``00``    | ADC_DATA    || ``sysClkOut`` | Traza Cascada |
|   ``01``    | ADC_DATA    || ``sysClk``    | Free Cascada  |
|   ``10``    | ADC_Counter || ``sysClkOut`` | Traza Counter |
|   ``11``    | ADC_Counter || ``sysClk``    | Free Counter  |

|    Base Address      |    AXI         ||   Size   |
|----------------------|----------------||----------|
|   ``0x4120_0000``    | AXI_control    || ``64K``  |
|   ``0x4200_0000``    | AXI_gencontrol || ``4K``   |
|   ``0x4121_0000``    | AXI_umbral     || ``64K``  |

# Umbral Dinámico
## Estas instrucciones establecen el umbral desde el prompt 

In [2]:
from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10
regset = MMIO(0x41210000, 0xC)
regset.write32(0,0xFF) # 
print(regset.read32(0x0))

255


In [3]:
regset = MMIO(0x41210000, 0xC)
print(regset.read32(0x08))

0


# gpioIN(5-4) 11 ADC_Counter sysClk   (FreeCounter)

In [4]:
TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30

modo = FreeCounter
# MMIO = Memory-mapped I/O(Input/Output) 
# MMIO almacena una dirección en un  espacio fisico dentro de la memoria ram, ademas la memoria y los registros 
#obtenidos por (I/O) son mapeados o asociados a valores de direccion

# dependiendo de la dirección se dara la orden de escribir o leer, esta seleccion dependera de la dirección que tenga como entrada 
# cuando accedemos a la dirección 4120 0000 estamos accediendo al axi de control 
# cuando accedemos a la dirección 4200 0000 estamos accediendo al axi de gencount
# con el comando write32 
from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10
regset = MMIO(0x41200000, 0xc)
regset.write32(0,modo | 0x08) # 


In [5]:
from periphery import MMIO
import numpy as np


TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30

modo = TrazaCounter

regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 

In [6]:
regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x01 ) # wr_en = 1

In [7]:
import time
regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # reset
regset.write32(0, modo | 0x01) # wr_en = 1
time.sleep(1)
regset.write32(0, modo | 0x02) # rd_en = 1

datos = []
for i in range(1030):
    regset.write32(0, modo | 0x02)# rd_en = 1
    regset.write32(0, modo | 0x06)# cntrl_clk rd_en = 1
    dato = 0x3FFF & regset.read32(0x08) # en mascara los 14 bits menos significativos
    datos.append(dato)
    print(hex(dato))


0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x116
0x117
0x118
0x119
0x11a
0x11b
0x11c
0x11d
0x11e
0x11f
0x120
0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x116
0x117
0x118
0x119
0x11a
0x11b
0x11c
0x11d
0x11e
0x11f
0x120
0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x116
0x117
0x118
0x119
0x11a
0x11b
0x11c
0x11d
0x11e
0x11f
0x120
0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x116
0x117
0x118
0x119
0x11a
0x11b
0x11c
0x11d
0x11e
0x11f
0x120
0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x116
0x117
0x118
0x119
0x11a
0x11b
0x11c
0x11d
0x11e
0x11f
0x120
0x101
0x102
0x103
0x104
0x105
0x106
0x10

# Propuesta Inicial leer la FIFO en forma sincrona
## regset = empty & full & 14BitsADC

In [8]:
from periphery import MMIO
import numpy as np

TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30

modo = TrazaCounter

ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10
regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 


In [9]:
import time
import matplotlib.pyplot as plt
%matplotlib inline
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val



In [10]:
import time
regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;  0x30 | 0x30 | 
TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30
modo = TrazaCounter

datos = []
for i in range(2):
    datos = []
    regset.write32(0, modo | 0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_ou 0x30 | t = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos signif 0x30 | icativos
        regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; w 0x30 | r_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    while(dato != 0x20000 ):
        regset.write32(0, modo | 0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0, modo | 0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    print("==================================================")
    print(i)
    
    


0x1c120
0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x116
0x117
0x118
0x119
0x11a
0x11b
0x11c
0x11d
0x11e
0x11f
0x120
0x4101
0x4102
0x4103
0x4104
0x4105
0x4106
0x4107
0x4108
0x4109
0x410a
0x410b
0x410c
0x410d
0x410e
0x410f
0x4110
0x4111
0x4112
0x4113
0x4114
0x4115
0x4116
0x4117
0x4118
0x4119
0x411a
0x411b
0x411c
0x411d
0x411e
0x411f
0x4120
0x8101
0x8102
0x8103
0x8104
0x8105
0x8106
0x8107
0x8108
0x8109
0x810a
0x810b
0x810c
0x810d
0x810e
0x810f
0x8110
0x8111
0x8112
0x8113
0x8114
0x8115
0x8116
0x8117
0x8118
0x8119
0x811a
0x811b
0x811c
0x811d
0x811e
0x811f
0x8120
0xc101
0xc102
0xc103
0xc104
0xc105
0xc106
0xc107
0xc108
0xc109
0xc10a
0xc10b
0xc10c
0xc10d
0xc10e
0xc10f
0xc110
0xc111
0xc112
0xc113
0xc114
0xc115
0xc116
0xc117
0xc118
0xc119
0xc11a
0xc11b
0xc11c
0xc11d
0xc11e
0xc11f
0xc120
0x101
0x102
0x103
0x104
0x105
0x106
0x107
0x108
0x109
0x10a
0x10b
0x10c
0x10d
0x10e
0x10f
0x110
0x111
0x112
0x113
0x114
0x115
0x

In [3]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30
modo = TrazaCounter

regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 


def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val


regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;

datos = []
for i in range(5):
    datos = []
    regset.write32(0, modo | 0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
        regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    intDatos = []
    while(dato != 0x20000 ):
        regset.write32(0, modo | 0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0, modo | 0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        #print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
        tmp = 0x3FFF & tmp # en mascara los 14 bits menos significativos
        valor=twos_comp(tmp,14)
        intDatos.append(valor)
    plt.figure(i)
    plt.plot(intDatos[0:500])
    plt.title('Prueba del Modo FreeCounter')
    plt.ylabel('Unidades ADC')
    plt.xlabel('Numero de muestras')
    #plt.stem(intDatos[10:164])
    print("==================================================")
    print(i)
    

KeyboardInterrupt: 

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30
modo = FreeCounter
#modo = TrazaCounter
regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 


def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val


regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;

datos = []
for i in range(5):
    datos = []
    regset.write32(0, modo | 0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
        regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    intDatos = []
    while(dato != 0x20000 ):
        regset.write32(0, modo | 0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0, modo | 0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        #print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
        tmp = 0x3FFF & tmp # en mascara los 14 bits menos significativos
        valor=twos_comp(tmp,14)
        intDatos.append(valor)
    plt.figure(i)
    plt.plot(intDatos[100:500])
    #plt.stem(intDatos[10:164])
    print("==================================================")
    print(i)
    

# Fija el umbral de adquisición

In [1]:
from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10
regset = MMIO(0x41210000, 0xC)
#regset.write32(0,0x210) #    <----------- Valor de umbral, disparo, trigger 0x210 = 528, 0x1FF = 511  0X208=520 0X190=400
regset.write32(0,0x90) #    <----------- Valor de umbral, disparo, trigger 0xBBB = 3000 0x1388 = 5000   0x104 = 260
resolucion = 0.1220703125 #si la escala de voltaje es de +-1 v
#resolucion = 0.06103515625  #si la escala de voltaje es de +1 v
voltaje = int(regset.read32(0x0))*resolucion 
print('en unidades adimensionales tenemos el valor de ')
print(regset.read32(0x0))
print('En escala de voltaje el valor correspondiente es ' + str(voltaje) + str(' mv'))

# el umbral de disparo podemos verlo por los valores maximos o minimos donde se seccionan los 16 puntos en el modo trazacouenter
# en hexadecimal el 260 es 0x104
#276 es 0x114
#290 es 0x122
#368 es 0x170
#400 es 0x190
#65  es 0x41
#96  es 0x60
#128 es 0x80
#80 es 0x50
#(FSVR)el rango de voltaje a escala completa es de +-1v (lv) y +- 20v (hv)
#la resolucion de un ADC se obtiene de la función 
#Resolucion = 1/2^{n}
#resolucion = FSVR/2^{n}
#resolucion = 2000 mv/2 ^{14}= 2000mv/16384 = 0.1220703125 mv

#hay que multiplicar la resolucion*valor de umbral para obtener el equivalente en mv
#el valor de disparo debe estar entre 30mv a 60mv esto es el rango del punto maximo de la señal


en unidades adimensionales tenemos el valor de 
144
En escala de voltaje el valor correspondiente es 17.578125 mv


In [ ]:
import time
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import numpy as np
from scipy.stats import norm
import statistics

from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30
#modo = TrazaCounter
modo = TrazaCascada
#modo = FreeCounter
#modo = FreeCascada
regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 

#definimos el nombre del archivo
#nombre_archivo = "toma_32muestras_detector_silicio4.csv"

#definimos la cantidad de columnas que se van a almacenar

#crear un dataframe vacio para almacenar los datos
df = pd.DataFrame()

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val


regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;

datos = []

i=0
while True:
    datos = []
    regset.write32(0, modo | 0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
        regset.write32(0,modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    intDatos = []   
    while(dato != 0x20000 ):
        regset.write32(0, modo | 0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0, modo | 0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        #print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
        tmp = 0x3FFF & tmp # en mascara los 14 bits menos significativos
        valor=twos_comp(tmp,14)
        intDatos.append(valor)
    
    #almacenar los datos de redpitaya en un archivo csv para lecturas externas en redpitaya
    df_datos =pd.DataFrame(intDatos)
    timestamp=int(time.time())
    df_datos.to_csv('datos_SiPM_64muestras_while_continuo_{}.csv'.format(timestamp), header=False, index=False)
    #i=+1
    
    
    #En este espacio estamos calculando la linea base del ruido, para ello e calcula la media, la desviacion estandar del 
    #corrimiento de datos
    std = np.std(intDatos[:])
    mu = np.mean(intDatos[:])
    
    print("================")
    
 

KeyboardInterrupt: 

### segmento de codigo de las graficas
### la toma de datos empezo 2:08 pm /2/junio/viernes/2022

### la toma de datos termino a las 11:22 am /5/junio/lunes/2022

In [ ]:
   
     #Los datos deben superar el valor del ruido 
    linea_base = mu + 2*std
    #impresion de valor de la media
    print("La media de la señal de ruido es:", mu)
    #impresion de valor de la media
    print("La desviación estándar de la señal es:", std)
    # Imprimir la línea base del ruido
    print("el umbral debe superar el valor del ruido con valor de :", linea_base)
   
    #############################################################################
     #presentacion de graficas plot, stem y el histograma de valores
    plt.figure(i)
  
    fig1,(ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
    ax1.plot(intDatos[:], color = 'g')
    #ax1.set_ylim(20,100)
    ax1.grid(True)
    ax1.set_xlabel('Numero de datos')
    ax1.set_ylabel('Unidades ADC')
    ax1.set_title('Perfiles de pulsos a 64 datos muestrasles')
    #ax2.plot(df[220:255],color ='purple')
    ax2.stem(intDatos[:],linefmt = 'g-',markerfmt = 'ko')
    #ax2.hist(intDatos[:], bins =16 )
    ax2.grid(True)
    #ax2.set_ylim(,)
    ax2.set_xlabel('Numero de datos')
    ax2.set_ylabel('Unidades ADC')
    ax2.set_title('Valores Discretos a 64 muestras')

    plt.show()
    print("==================================================")
    print(i)

# este es es un ejemplo de como graficar el fenomeno de distintas formas 

In [ ]:
import time
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import numpy as np
from scipy.stats import norm

from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30
#modo = TrazaCounter
#modo = TrazaCascada
#modo = FreeCounter
modo = FreeCascada
regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 


def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val


regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;

datos = []
for i in range(5):
    datos = []
    regset.write32(0, modo | 0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
        regset.write32(0,modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    intDatos = []
    while(dato != 0x20000 ):
        regset.write32(0, modo | 0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0, modo | 0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        #print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
        tmp = 0x3FFF & tmp # en mascara los 14 bits menos significativos
        valor=twos_comp(tmp,14)
        intDatos.append(valor)
    #almacenar los datos de redpitaya en un archivo csv para lecturas externas en redpitaya
    df_datos =pd.DataFrame(intDatos)
    df_datos.to_csv('datos_ruido_sistema.csv', header=True, index=False)
    #--------------------------
    plt.figure(i)
    fig =plt.figure()
    plt.plot(intDatos[:])
    plt.title('ruido') 
    plt.xlabel('numero de datos')
    plt.ylabel('Unidades Adimensionales')
    #_------------------------------------------
    fig1 = plt.figure()
    plt.hist(intDatos[:],bins=20)
    plt.title('histograma')
    plt.xlabel('numero de datos')
    plt.ylabel('Unidades ADC')
    #---------------------------------------------------------------
    fig2 = plt.figure()
    plt.title('tren de 16 puntos')
    plt.xlabel('numero de datos')
    plt.stem(intDatos[0:100])
    plt.show()
    print("==================================================")
    print(i)
    

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

TrazaCascada = 0x00
FreeCascada  = 0x10
TrazaCounter = 0x20
FreeCounter  = 0x30
modo = TrazaCounter
#modo = TrazaCascada
#modo = FreeCounter
#modo = FreeCascada

regset = MMIO(0x41200000, 0xc)
regset.write32(0, modo | 0x08) # 


def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val


regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;

datos = []
for i in range(5):
    datos = []
    regset.write32(0, modo | 0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
        regset.write32(0, modo | 0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    intDatos = []
    while(dato != 0x20000 ):
        regset.write32(0, modo | 0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0, modo | 0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        #print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
        tmp = 0x3FFF & tmp # en mascara los 14 bits menos significativos
        valor=twos_comp(tmp,14)
        intDatos.append(valor)
    plt.figure(i)
    plt.plot(intDatos[:])
    plt.show()
    #plt.stem(intDatos[10:164])
    print("==================================================")
    print(i)

In [ ]:
twos_comp(0x100,14)

# Hasta acá revisado

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10
regset = MMIO(0x41200000, 0xc)
regset.write32(0,0x08) # 


def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val


regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
#regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
#time.sleep(1)
#regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;

datos = []
for i in range(5):
    datos = []
    regset.write32(0,0x08) # nrst = 1; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 0;
    regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;   
    #regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
    #regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
    dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
    while(dato != 0x10000 ):
        dato = 0x10000 & regset.read32(0x08) # en mascara los 14 bits menos significativos
        regset.write32(0,0x01) # nrst = 0; cntrl_clk = 0; rd_en_out = 0; wr_en_out = 1;
    
    tmp = regset.read32(0x08)
    dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
    intDatos = []
    while(dato != 0x20000 ):
        regset.write32(0,0x02) # nrst = 0; cntrl_clk = 0; rd_en_out = 1; wr_en_out = 0;
        regset.write32(0,0x06) # nrst = 0; cntrl_clk = 1; rd_en_out = 1; wr_en_out = 0;
        datos.append(tmp)
        #print(hex(tmp))
        tmp = regset.read32(0x08)
        dato = 0x20000 & tmp # en mascara los 14 bits menos significativos
        tmp = 0x3FFF & tmp # en mascara los 14 bits menos significativos
        valor=twos_comp(tmp,14)
        intDatos.append(valor)
    plt.figure(i)
    plt.plot(intDatos[128:256])
    plt.stem(intDatos[128:256])
    print("==================================================")
    print(i)
    

In [ ]:
import time

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(intDatos[1:640])
plt.stem(intDatos[1:640])

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

intDatos = []
for i in range(len(datos)):
    valor=twos_comp(datos[i],14)
    intDatos.append(valor)
plt.plot(intDatos[1:90])
plt.stem(intDatos[1:90])
plt.title("Señal de prueba a 10MHz (100 ns)")


In [ ]:
import time
regset = MMIO(0x41200000, 0xc)
#regset.write32(0,0x08) # reset
#regset.write32(0,0x01) # wr_en = 1
regset.write32(0,0x08) # reset
regset.write32(0,0x01) # wr_en = 1
time.sleep(2)
regset.write32(0,0x02) # rd_en = 1

datos = []
for i in range(8192):
    regset.write32(0,0x02)# rd_en = 1
    regset.write32(0,0x06)# cntrl_clk rd_en = 1
    dato = 0x3FFF & regset.read32(0x08)
    datos.append(dato)
    print(hex(dato))

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

intDatos = []
for i in range(len(datos)):
    valor=twos_comp(datos[i],14)
    intDatos.append(valor/8192)
plt.plot(intDatos)
plt.stem(intDatos)
plt.title("Señal de prueba a 10MHz (100 ns)")

In [ ]:
plt.plot(intDatos[128:512])
plt.stem(intDatos[128:512])
plt.title("Señal de prueba a 10MHz (100 ns)")

In [ ]:
datos[0]

In [ ]:
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val 

for i in range(16):
    intDecimal = twos_comp(i, 4)
    print('{} {}'.format(i,intDecimal) )


In [ ]:
import time
regset = MMIO(0x41200000, 0xc)
regset.write32(0,0x01) # wr_en = 1
regset.write32(0,0x09) # nrst = 1
regset.write32(0,0x01) # wr_en = 1
time.sleep(1)

for i in range(20):
    regset.write32(0,0x02)# rd_en = 1
    regset.write32(0,0x06)# cntrl_clk rd_en = 1
    print(hex(regset.read32(0x08)))



In [ ]:
import time
regset = MMIO(0x41200000, 0xc)
regset.write32(0,0x09) # nrst = 1
regset.write32(0,0x01) # wr_en = 1
regset.write32(0,0x09) # nrst = 1
regset.write32(0,0x01) # wr_en = 1
#time.sleep(1)
datos=[]
for i in range(100):
    #print(regsetAXI_CONTROL.read32(0x08))
    regset.write32(0,0x02)# rd_en = 1
    regset.write32(0,0x06)# cntrl_clk rd_en = 1
    print(hex(regset.read32(0x08)))




In [ ]:
import time

regsetAXI_CONTROL = MMIO(0x42000000, 0xc)
regsetAXI_CONTROL.write32(0,0x02) # SCLR counter
regsetAXI_CONTROL.write32(0,0x01) # ENA counter

regset = MMIO(0x41200000, 0xc)
regset.write32(0,0x01) # wr_en = 1
regset.write32(0,0x09) # nrst = 1
regset.write32(0,0x01) # wr_en = 1
time.sleep(1)

for i in range(20):
    regset.write32(0,0x02)# rd_en = 1
    regset.write32(0,0x06)# cntrl_clk rd_en = 1
    print(hex(0x3FFF & regset.read32(0x08)))
    
    

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

regsetAXI_CONTROL = MMIO(0x42000000, 0xc)
regsetAXI_CONTROL.write32(0,0x02) # SCLR counter
regsetAXI_CONTROL.write32(0,0x01) # ENA counter

regset = MMIO(0x41200000, 0xc)
regset.write32(0,0x01) # wr_en = 1
regset.write32(0,0x09) # nrst = 1
regset.write32(0,0x01) # wr_en = 1

while( not(regset.read32(0x08) & 0x010000) == 0x010000): # wait full == 1
    break

i = 0
datos=[]
while( not(regset.read32(0x08) & 0x020000) == 0x020000): # wait empty == 1
    regset.write32(0,0x02)# rd_en = 1
    regset.write32(0,0x06)# cntrl_clk rd_en = 1
    i += 1
    datos.append(0xFFFF & regset.read32(0x08))
    #print(hex(0xFFFF & regset.read32(0x08)))
plt.plot(datos[1:128])
#print("= = = = = =  = =")
#print(i)

In [ ]:
plt.plot(datos[1:128])


In [ ]:
#name = 'Niroshan'
#age  = 25;
#print(f"Hello I'm {name} and {age} years young")

# Toma de datos con el generador de pulso Aligent 81101A

# proyecto vidado localizado en D:\redpitaya\tmp\3_stopwatch_modif\readFIFO_ADC
# archivo bitstream readfifo_adc3.bit, localizado en /root/RedPitaya
## ejecutar antes de probar el código python
## cat readfifo_adc3.bit > /dev/xdevcfg

In [ ]:
# MMIO = Memory-mapped I/O(Input/Output) 
# MMIO almacena una dirección en un  espacio fisico dentro de la memoria ram, ademas la memoria y los registros 
#obtenidos por (I/O) son mapeados o asociados a valores de direccion

# dependiendo de la dirección se dara la orden de escribir o leer, esta seleccion dependera de la dirección que tenga como entrada 
# cuando accedemos a la dirección 4120 0000 estamos accediendo al axi de control 
# cuando accedemos a la dirección 4200 0000 estamos accediendo al axi de gencount
# con el comando write32 
from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x10

regset = MMIO(0x41200000, 0xc)

regset.write32(0,0x08) # reset Observar que responda la tarjeta

In [ ]:
# Adquiere un par de datos
import time
regset = MMIO(0x41200000, 0xc)
regset.write32(0,0x08) # reset
regset.write32(0,0x01) # wr_en = 1
time.sleep(1)
regset.write32(0,0x02) # rd_en = 1

datos = []
for i in range(100):
    regset.write32(0,0x02)# rd_en = 1
    regset.write32(0,0x06)# cntrl_clk rd_en = 1
    dato = 0x3FFF & regset.read32(0x08)
    datos.append(dato)
    print(hex(dato))


In [ ]:
import matplotlib.pyplot as plt

# Generar algunos datos de ejemplo
x = [i for i in range(100)]
y = [i ** 2 for i in range(100)]

# Crear el gráfico
fig, ax = plt.subplots()
ax.plot(x, y)

# Habilitar la herramienta de zoom
ax.set_title('Haz zoom en el gráfico')
ax.set_xlabel('Eje X')
ax.set_ylabel('Eje Y')
ax.set_xlim([0, 100])
ax.set_ylim([0, 10000])
ax.set_autoscale_on(False)  # desactivar el escalado automático para que el zoom sea más preciso
ax.grid(True)
ax.set_axisbelow(True)
zoom = plt.axes([0.2, 0.5, 0.25, 0.25])  # crear un nuevo eje para la herramienta de zoom
axzoom = plt.axes()  # crear otro eje para almacenar el área de zoom

# Agregar la herramienta de zoom al gráfico
zoom_button = plt.widgets.Button(zoom, '+')
def zoom_in(event):
    axzoom.set_xlim([20, 40])
    axzoom.set_ylim([500, 2000])
zoom_button.on_clicked(zoom_in)

# Mostrar el gráfico
plt.show()

# Pruebas de código

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val



In [ ]:
import time
import matplotlib.pyplot as plt
data=[]
data1=[]
for i in range(16):
    val = twos_comp(twos_comp(i, 4),4)
    val1 = twos_comp(i,4)
    data.append(val)
    data1.append(val1)
plt.plot(data)
plt.plot(data1)

In [ ]:
a=twos_comp(-3,4)
b=twos_comp(-2,4)

if a > b :
    print('a es mayor')
else:
    print('b es mayor')

# Fija el umbral de adquisición

|    Base Address      |    AXI         ||   Size   |
|----------------------|----------------||----------|
|   ``0x4120_0000``    | AXI_control    || ``64K``  |
|   ``0x4200_0000``    | AXI_gencontrol || ``4K``   |
|   ``0x4121_0000``    | AXI_umbral     || ``64K``  |

In [ ]:
from enum import Enum
import numpy as np
import json
import os
#from pynq import DefaultIP
#from pynq.ps import CPU_ARCH, ZU_ARCH
import struct

def _float2int(value: float) -> int:
    """Pack a single precision floating point into a 32-bit integer"""
    return int.from_bytes(struct.pack('f', np.single(value)), 'little')


In [ ]:
hex(_float2int(-1.0))

In [ ]:
hex(_float2int(1.0))

In [ ]:
from periphery import MMIO
import numpy as np
ADDRESS_RANGE = 0x10
ADDRESS_OFFSET = 0x10
regset = MMIO(0x41210000, ADDRESS_RANGE)
regset.write32(0,0xFFFF) # 
regset.write32(1,0x3f80) # 

print(regset.read32(0x0000))
print(regset.read32(0x0001))


In [ ]:
variable = -1.0
direccion=hex(id(variable))
print(direccion)
map(direccion)


In [ ]:
from periphery import MMIO
import numpy as np
IP_BASE_ADDRESS = 0x41210000
ADDRESS_RANGE = 0x1000
ADDRESS_OFFSET = 0x04

mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

data = 0xABCD1234
mmio.write32(ADDRESS_OFFSET, data)
for i in range(0x10):
    #result = mmio.read32(ADDRESS_OFFSET)
    #print(hex(result))
    result = mmio.read32(i)
    print(hex(result))


In [ ]:
from enum import Enum
import numpy as np
import json
import os
#from pynq import DefaultIP
#from pynq.ps import CPU_ARCH, ZU_ARCH
import struct

def _float2int(value: float) -> int:
    """Pack a single precision floating point into a 32-bit integer"""
    return int.from_bytes(struct.pack('f', np.single(value)), 'little')